<a href="https://colab.research.google.com/github/Abdu1964/NeuralNetwork/blob/main/AI_Model_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Load dataset

In [ ]:
# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# %% [markdown]
# ## 2. Logistic Regression (scikit-learn)
# %%
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score,
                           recall_score, f1_score, confusion_matrix)


Training samples: 455, Test samples: 114


# Initialize and train

In [ ]:
logreg = LogisticRegression(
    penalty='l2',    # Regularization type
    C=1.0,          # Inverse regularization strength
    max_iter=1000,   # Maximum iterations
    random_state=42
)
logreg.fit(X_train_scaled, y_train)

# Predictions
y_pred = logreg.predict(X_test_scaled)


# Evaluation

In [ ]:
def print_metrics(y_true, y_pred):
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

print("Logistic Regression Performance:")
print_metrics(y_test, y_pred)

Logistic Regression Performance:
Accuracy: 0.9737
Precision: 0.9722
Recall: 0.9859
F1 Score: 0.9790
Confusion Matrix:
[[41  2]
 [ 1 70]]


#  Neural Network (PyTorch)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors

In [ ]:
train_data = TensorDataset(
    torch.FloatTensor(X_train_scaled),  # Features as float32
    torch.LongTensor(y_train)           # Labels as int64
)
test_data = TensorDataset(
    torch.FloatTensor(X_test_scaled),
    torch.LongTensor(y_test)
)

# Data loaders

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


# Define model

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.layers(x)


# Initialize

In [ ]:
model = NeuralNet(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop

In [ ]:
for epoch in range(50):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 10, Loss: 0.0661
Epoch 20, Loss: 0.0020
Epoch 30, Loss: 0.0002
Epoch 40, Loss: 0.0002
Epoch 50, Loss: 0.0035


# Evaluation

In [ ]:
model.eval()
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())

print("\nNeural Network Performance:")
print_metrics(y_test, all_preds)


Neural Network Performance:
Accuracy: 0.9825
Precision: 0.9859
Recall: 0.9859
F1 Score: 0.9859
Confusion Matrix:
[[42  1]
 [ 1 70]]


In [ ]:
# Example: Mapping results to class labels
class_names = ['Benign', 'Malignant']
predictions = [class_names[pred] for pred in all_preds]

# Print a few results
for i in range(10):  # Displaying first 10 predictions
    print(f"Predicted: {predictions[i]}, Actual: {class_names[y_test[i]]}")
